# Project Note Book

In this notebook we have the entire pipeline of the project. Here, we use the models from [Hugging Face](https://huggingface.co/nlp-group-6) to be able to run our pipline. But if you would like to see how we trained the models, you can check the "notebooks" folder in the repository.

## Pipeline

Our pipeline consists of the following steps:
1. Generate the question given the context and the answer.
2. Generate the disractor given the question and the answer.

This can be outline in image below:

![pipeline](images/pipeline.png)

First we import the transformers library from Hugging Face and the models we will use in the pipeline.

In [1]:
from transformers import BartTokenizer, BartForConditionalGeneration

Then we start with the question generator model.

In [2]:
question_model = BartForConditionalGeneration.from_pretrained('nlp-group-6/sciq-question-generator')
question_tokenizer = BartTokenizer.from_pretrained('nlp-group-6/sciq-question-generator')

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Finally, we load the disractor generator model.

In [3]:
options_model = BartForConditionalGeneration.from_pretrained('nlp-group-6/sciq-options-generator-generated-questions')
options_tokenizer = BartTokenizer.from_pretrained('nlp-group-6/sciq-options-generator-generated-questions')

Now we put the pipeline together and test it with a simple example.

In [4]:
support = "Milk has a white color."

# generate the question
inputs = question_tokenizer([support], return_tensors='pt', max_length=512, truncation=True)
question = question_model.generate(**inputs)
question = question_tokenizer.decode(question[0], skip_special_tokens=True)

correct_answer = "white"

# generate the options
inputs = options_tokenizer([support], return_tensors='pt', max_length=512, truncation=True)
options = options_model.generate(**inputs, num_return_sequences=4, num_beams=4, max_length=50)
options = options_tokenizer.batch_decode(options, skip_special_tokens=True)

print(question)
print(correct_answer)
print(options)

/home/roben/Documents/courses/NLP/Science-Quiz-Generation/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


What color is milk?
white
['Milk has a blue color', 'Milk has a black color', 'Milk has a yellow color', 'milk has a blue color']
